# 用户参与度与情感传播分析

本notebook主要完成以下任务：
1. 基于点赞数的用户分层分析
2. 高人气推文与普通推文的情感质量对比
3. 核心用户与外围用户的情感传播分析

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 设置图形风格
plt.style.use('seaborn')

## 1. 数据加载与预处理

In [ ]:
# 加载数据
df = pd.read_csv("../2_data/finbert_scored_tweets.csv", parse_dates=["created_at"])

# 处理点赞数
def extract_likes(likes_dict):
    if pd.isna(likes_dict):
        return 0
    try:
        if isinstance(likes_dict, str):
            import ast
            likes_dict = ast.literal_eval(likes_dict)
        return likes_dict.get('total', 0)
    except:
        return 0

df['likes'] = df['likes'].apply(extract_likes)

# 处理用户信息
def extract_username(user_dict):
    if pd.isna(user_dict):
        return 'unknown'
    try:
        if isinstance(user_dict, str):
            import ast
            user_dict = ast.literal_eval(user_dict)
        return user_dict.get('username', 'unknown')
    except:
        return 'unknown'

df['username'] = df['user'].apply(extract_username)

print(f"总推文数: {len(df)}")
print(f"总用户数: {df['username'].nunique()}")
print(f"总点赞数: {df['likes'].sum()}")

## 2. 基于点赞数的用户分层分析

In [ ]:
# 计算用户总点赞数
user_engagement = df.groupby('username').agg({
    'likes': 'sum',
    'finbert_score': ['mean', 'std', 'count']
}).reset_index()

user_engagement.columns = ['username', 'total_likes', 'avg_sentiment', 'sentiment_std', 'tweet_count']

# 定义用户分层标准
like_threshold = user_engagement['total_likes'].quantile(0.95)  # 前5%的用户
user_engagement['user_type'] = np.where(user_engagement['total_likes'] >= like_threshold, '高人气用户', '普通用户')

# 展示用户分层统计
print("用户分层统计:")
print(user_engagement.groupby('user_type').agg({
    'username': 'count',
    'total_likes': 'mean',
    'avg_sentiment': 'mean',
    'tweet_count': 'mean'
}).round(2))

# 可视化用户分层
plt.figure(figsize=(12, 6))
sns.boxplot(x='user_type', y='avg_sentiment', data=user_engagement)
plt.title('不同用户类型的情感分布')
plt.xlabel('用户类型')
plt.ylabel('平均情感分数')
plt.show()

## 3. 高人气推文与普通推文的情感质量对比

In [ ]:
# 定义高人气推文标准
tweet_like_threshold = df['likes'].quantile(0.95)  # 前5%的推文
df['tweet_type'] = np.where(df['likes'] >= tweet_like_threshold, '高人气推文', '普通推文')

# 计算情感统计
tweet_sentiment_stats = df.groupby('tweet_type').agg({
    'finbert_score': ['count', 'mean', 'std', 'min', 'max'],
    'likes': 'mean'
}).round(4)

print("推文情感统计:")
print(tweet_sentiment_stats)

# 可视化情感分布
plt.figure(figsize=(12, 6))
sns.violinplot(x='tweet_type', y='finbert_score', data=df)
plt.title('高人气推文与普通推文的情感分布')
plt.xlabel('推文类型')
plt.ylabel('情感分数')
plt.show()

## 4. 情感传播网络分析

In [ ]:
# 创建用户交互网络
def create_interaction_network(df, time_window=timedelta(hours=24)):
    G = nx.DiGraph()
    
    # 按时间排序
    df = df.sort_values('created_at')
    
    # 创建用户节点
    for username in df['username'].unique():
        user_sentiment = df[df['username'] == username]['finbert_score'].mean()
        G.add_node(username, sentiment=user_sentiment)
    
    # 创建边（用户交互）
    for i, row in df.iterrows():
        current_time = row['created_at']
        window_start = current_time - time_window
        
        # 查找时间窗口内的其他推文
        window_tweets = df[(df['created_at'] >= window_start) & 
                          (df['created_at'] <= current_time)]
        
        # 添加边
        for _, other_row in window_tweets.iterrows():
            if row['username'] != other_row['username']:
                G.add_edge(other_row['username'], row['username'])
    
    return G

# 创建网络
G = create_interaction_network(df)

# 计算网络指标
print("网络统计:")
print(f"节点数: {G.number_of_nodes()}")
print(f"边数: {G.number_of_edges()}")

# 识别核心用户（使用PageRank算法）
pagerank = nx.pagerank(G)
core_users = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]

print("\n核心用户 (PageRank排名前10):")
for user, score in core_users:
    print(f"{user}: {score:.4f}")

# 可视化网络
plt.figure(figsize=(15, 15))
pos = nx.spring_layout(G, k=1, iterations=50)

# 绘制节点
node_colors = [G.nodes[node]['sentiment'] for node in G.nodes()]
node_sizes = [pagerank[node] * 10000 for node in G.nodes()]

nx.draw_networkx_nodes(G, pos, 
                      node_color=node_colors,
                      node_size=node_sizes,
                      cmap='RdYlBu',
                      alpha=0.6)

# 绘制边
nx.draw_networkx_edges(G, pos, alpha=0.2)

# 添加核心用户标签
core_user_labels = {user: user for user, _ in core_users}
nx.draw_networkx_labels(G, pos, labels=core_user_labels)

plt.title('用户情感传播网络\n(节点大小表示影响力，颜色表示情感倾向)')
plt.colorbar(plt.cm.ScalarMappable(cmap='RdYlBu'), label='情感分数')
plt.axis('off')
plt.show()

## 5. 保存分析结果

In [ ]:
# 保存用户分层结果
user_engagement.to_csv("../2_data/user_engagement_analysis.csv", index=False)

# 保存推文分类结果
df[['username', 'created_at', 'likes', 'finbert_score', 'tweet_type']].to_csv("../2_data/tweet_engagement_analysis.csv", index=False)

print("分析结果已保存到文件")